# T1562.004 - Disable or Modify System Firewall
Adversaries may disable or modify system firewalls in order to bypass controls limiting network usage. Changes could be disabling the entire mechanism as well as adding, deleting, or modifying particular rules. This can be done numerous ways depending on the operating system, including via command-line, editing Windows Registry keys, and Windows Control Panel.

Modifying or disabling a system firewall may enable adversary C2 communications, lateral movement, and/or data exfiltration that would otherwise not be allowed. 

## Atomic Tests

### Atomic Test #1 - Disable Microsoft Defender FirewallDisables the Microsoft Defender Firewall for the current profile.
Caution if you access remotely the host where the test runs! Especially with the cleanup command which will re-enable firewall for the current profile...
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
netsh advfirewall set currentprofile state off
```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 1

#### Cleanup: 
```cmd
netsh advfirewall set currentprofile state on >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Disable Microsoft Defender Firewall via RegistryDisables the Microsoft Defender Firewall for the public profile via registry
Caution if you access remotely the host where the test runs! Especially with the cleanup command which will re-enable firewall for the current profile...
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_LOCAL_MACHINE\SYSTEM\CurrentControlSet\Services\SharedAccess\Parameters\FirewallPolicy\PublicProfile" /v "EnableFirewall" /t REG_DWORD /d 0 /f
```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 2

#### Cleanup: 
```cmd
reg add "HKEY_LOCAL_MACHINE\SYSTEM\CurrentControlSet\Services\SharedAccess\Parameters\FirewallPolicy\PublicProfile" /v "EnableFirewall" /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 2 -Cleanup

### Atomic Test #3 - Allow SMB and RDP on Microsoft Defender FirewallAllow all SMB and RDP rules on the Microsoft Defender Firewall for all profiles.
Caution if you access remotely the host where the test runs! Especially with the cleanup command which will reset the firewall and risk disabling those services...
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
netsh advfirewall firewall set rule group="remote desktop" new enable=Yes
netsh advfirewall firewall set rule group="file and printer sharing" new enable=Yes
```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 3

#### Cleanup: 
```cmd
netsh advfirewall reset >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 3 -Cleanup

### Atomic Test #4 - Opening ports for proxy - HARDRAINThis test creates a listening interface on a victim device. This tactic was used by HARDRAIN for proxying.

reference: https://www.us-cert.gov/sites/default/files/publications/MAR-10135536-F.pdf
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
netsh advfirewall firewall add rule name="atomic testing" action=allow dir=in protocol=TCP localport=450 
```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 4

#### Cleanup: 
```cmd
netsh advfirewall firewall delete rule name="atomic testing" protocol=TCP localport=450 >nul 2>&1```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 4 -Cleanup

### Atomic Test #5 - Open a local port through Windows Firewall to any profileThis test will attempt to open a local port defined by input arguments to any profile**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
netsh advfirewall firewall add rule name="Open Port to Any" dir=in protocol=tcp localport=3389 action=allow profile=any```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 5

#### Cleanup: 
```powershell
netsh advfirewall firewall delete rule name="Open Port to Any" | Out-Null```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 5 -Cleanup

### Atomic Test #6 - Allow Executable Through Firewall Located in Non-Standard LocationThis test will attempt to allow an executable through the system firewall located in the Users directory**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
Copy-Item PathToAtomicsFolder\T1562.004\bin\AtomicTest.exe -Destination "C:\Users\$env:UserName" -Force
netsh advfirewall firewall add rule name="Atomic Test" dir=in action=allow program="C:\Users\$env:UserName\AtomicTest.exe" enable=yes
```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 6

#### Cleanup: 
```powershell
netsh advfirewall firewall delete rule name="Atomic Test" | Out-Null
Remove-Item C:\Users\$env:UserName\AtomicTest.exe -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 6 -Cleanup

### Atomic Test #7 - Stop/Start UFW firewall
Stop the Uncomplicated Firewall (UFW) if installed.

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: Check if ufw is installed on the machine.

##### Check Prereq Commands:
```sh
if [ ! -x "$(command -v ufw)" ]; then echo -e "\n***** ufw NOT installed *****\n"; exit 1; fi
if echo "$(ufw status)" |grep -q "inactive"; then echo -e "\n***** ufw inactive *****\n"; exit 1; fi 

```
##### Get Prereq Commands:
```sh
echo ""

```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 7 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
ufw disable
```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 7

#### Cleanup: 
```sh
ufw enable
ufw status verbose
```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 7 -Cleanup

### Atomic Test #8 - Stop/Start UFW firewall systemctl
Stop the Uncomplicated Firewall (UFW) if installed, using systemctl. 

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: Check if systemctl and ufw is installed on the machine.

##### Check Prereq Commands:
```sh
if [ ! -x "$(command -v systemctl)" ]; then echo -e "\n***** systemctl NOT installed *****\n"; exit 1; fi
if [ ! -x "$(command -v ufw)" ]; then echo -e "\n***** ufw NOT installed *****\n"; exit 1; fi
if echo "$(ufw status)" |grep -q "inactive"; then echo -e "\n***** ufw inactive *****\n"; exit 1; fi 

```
##### Get Prereq Commands:
```sh
echo ""

```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 8 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
systemctl stop ufw
```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 8

#### Cleanup: 
```sh
systemctl start ufw
systemctl status ufw
```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 8 -Cleanup

### Atomic Test #9 - Turn off UFW logging
Turn off the Uncomplicated Firewall (UFW) logging. 

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: Check if ufw is installed on the machine and enabled.

##### Check Prereq Commands:
```sh
if [ ! -x "$(command -v ufw)" ]; then echo -e "\n***** ufw NOT installed *****\n"; exit 1; fi
if echo "$(ufw status)" |grep -q "inactive"; then echo -e "\n***** ufw inactive *****\n"; exit 1; fi 

```
##### Get Prereq Commands:
```sh
echo ""

```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 9 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
ufw logging off
```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 9

#### Cleanup: 
```sh
ufw logging low
ufw status verbose
```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 9 -Cleanup

### Atomic Test #10 - Add and delete UFW firewall rules
Add and delete a rule on the Uncomplicated Firewall (UFW) if installed and enabled. 

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: Check if ufw is installed on the machine and enabled.

##### Check Prereq Commands:
```sh
if [ ! -x "$(command -v ufw)" ]; then echo -e "\n***** ufw NOT installed *****\n"; exit 1; fi
if echo "$(ufw status)" |grep -q "inactive"; then echo -e "\n***** ufw inactive *****\n"; exit 1; fi 

```
##### Get Prereq Commands:
```sh
echo ""

```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 10 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
ufw prepend deny from 1.2.3.4
ufw status numbered
```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 10

#### Cleanup: 
```sh
{ echo y; echo response; } | ufw delete 1
ufw status numbered
```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 10 -Cleanup

### Atomic Test #11 - Edit UFW firewall user.rules file
Edit the Uncomplicated Firewall (UFW) rules file /etc/ufw/user.rules.

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: Check if /etc/ufw/user.rules exists.

##### Check Prereq Commands:
```sh
if [ ! -f "/etc/ufw/user.rules" ]; then echo -e "\n***** ufw NOT installed *****\n"; exit 1; fi

```
##### Get Prereq Commands:
```sh
echo ""

```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 11 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
echo "# THIS IS A COMMENT" >> /etc/ufw/user.rules
grep "# THIS IS A COMMENT" /etc/ufw/user.rules
```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 11

#### Cleanup: 
```sh
sed -i 's/# THIS IS A COMMENT//g' /etc/ufw/user.rules
```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 11 -Cleanup

### Atomic Test #12 - Edit UFW firewall ufw.conf file
Edit the Uncomplicated Firewall (UFW) configuration file /etc/ufw/ufw.conf 
which controls if the firewall starts on boot and its logging level.

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: Check if /etc/ufw/ufw.conf exists.

##### Check Prereq Commands:
```sh
if [ ! -f "/etc/ufw/ufw.conf" ]; then echo -e "\n***** ufw NOT installed *****\n"; exit 1; fi

```
##### Get Prereq Commands:
```sh
echo ""

```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 12 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
echo "# THIS IS A COMMENT" >> /etc/ufw/ufw.conf
grep "# THIS IS A COMMENT" /etc/ufw/ufw.conf
```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 12

#### Cleanup: 
```sh
sed -i 's/# THIS IS A COMMENT//g' /etc/ufw/ufw.conf
cat /etc/ufw/ufw.conf
```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 12 -Cleanup

### Atomic Test #13 - Edit UFW firewall sysctl.conf file
Edit the Uncomplicated Firewall (UFW) configuration file for setting network 
variables /etc/ufw/sysctl.conf.

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: Check if /etc/ufw/sysctl.conf exists.

##### Check Prereq Commands:
```sh
if [ ! -f "/etc/ufw/sysctl.conf" ]; then echo -e "\n***** ufw NOT installed *****\n"; exit 1; fi

```
##### Get Prereq Commands:
```sh
echo ""

```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 13 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
echo "# THIS IS A COMMENT" >> /etc/ufw/sysctl.conf
grep "# THIS IS A COMMENT" /etc/ufw/sysctl.conf
```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 13

#### Cleanup: 
```sh
sed -i 's/# THIS IS A COMMENT//g' /etc/ufw/sysctl.conf
cat /etc/ufw/sysctl.conf
```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 13 -Cleanup

### Atomic Test #14 - Edit UFW firewall main configuration file
Edit the Uncomplicated Firewall (UFW) main configuration file for setting 
default policies /etc/default/ufw.

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: Check if /etc/default/ufw exists.

##### Check Prereq Commands:
```sh
if [ ! -f "/etc/default/ufw" ]; then echo -e "\n***** ufw NOT installed *****\n"; exit 1; fi

```
##### Get Prereq Commands:
```sh
echo ""

```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 14 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
echo "# THIS IS A COMMENT" >> /etc/default/ufw
grep "# THIS IS A COMMENT" /etc/default/ufw
```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 14

#### Cleanup: 
```sh
sed -i 's/# THIS IS A COMMENT//g' /etc/default/ufw
```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 14 -Cleanup

### Atomic Test #15 - Tail the UFW firewall log file
Print  the last 10 lines of the Uncomplicated Firewall (UFW) log file 
/var/log/ufw.log.

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: Check if /var/log/ufw.log exists.

##### Check Prereq Commands:
```sh
if [ ! -f "/var/log/ufw.log" ]; then echo -e "\n***** ufw NOT logging *****\n"; exit 1; fi

```
##### Get Prereq Commands:
```sh
echo ""

```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 15 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
tail /var/log/ufw.log
```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 15

### Atomic Test #16 - LockBit Black - Unusual Windows firewall registry modification -cmdAn adversary tries to modify the windows firewall registry
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKLM\SOFTWARE\Policies\Microsoft\WindowsFirewall\DomainProfile" /v EnableFirewall /t REG_DWORD /d 0 /f
reg add "HKLM\SOFTWARE\Policies\Microsoft\WindowsFirewall\StandardProfile" /v EnableFirewall /t REG_DWORD /d 0 /f
```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 16

#### Cleanup: 
```cmd
reg delete "HKLM\SOFTWARE\Policies\Microsoft\WindowsFirewall\DomainProfile" /v EnableFirewall /f >nul 2>&1
reg delete "HKLM\SOFTWARE\Policies\Microsoft\WindowsFirewall\StandardProfile" /v EnableFirewall /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 16 -Cleanup

### Atomic Test #17 - LockBit Black - Unusual Windows firewall registry modification -PowershellAn adversary tries to modify the windows firewall registry.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
New-ItemProperty "HKLM:\SOFTWARE\Policies\Microsoft\WindowsFirewall\DomainProfile" -Name EnableFirewall -PropertyType DWORD -Value 0 -Force
New-ItemProperty "HKLM:\SOFTWARE\Policies\Microsoft\WindowsFirewall\StandardProfile" -Name EnableFirewall -PropertyType DWORD -Value 0 -Force
```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 17

#### Cleanup: 
```powershell
Remove-ItemProperty "HKLM:\SOFTWARE\Policies\Microsoft\WindowsFirewall\DomainProfile"  -Name EnableFirewall -Force -ErrorAction Ignore
Remove-ItemProperty "HKLM:\SOFTWARE\Policies\Microsoft\WindowsFirewall\StandardProfile" -Name EnableFirewall -Force -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 17 -Cleanup

### Atomic Test #18 - Blackbit - Disable Windows Firewall using netsh firewallAn adversary tries to modify the windows firewall configuration using the deprecated netsh firewall command (command still works).     
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
netsh firewall set opmode mode=disable
```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 18

#### Cleanup: 
```cmd
netsh firewall set opmode mode=enable >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1562.004 -TestNumbers 18 -Cleanup

## Detection
Monitor processes and command-line arguments to see if firewalls are disabled or modified. Monitor Registry edits to keys that manage firewalls.